# Code Generator

Convert the program from Python to high performance C++ code.

In [1]:
import os 
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
from IPython.display import Markdown, display 

In [2]:
# load the environment variables 
load_dotenv(override=True)

True

# Pricing of GPT-5 model
```
Input:          $1.250 / 1M tokens
Cached input:   $0.125 / 1M tokens
Output:         $10.000 / 1M tokens

In [3]:
# get openai key 
api_key=os.getenv("OPENAI_API_KEY")

# create openai client object 
openai=OpenAI()

# models 
OPENAI_MODEL="gpt-5"

# Get system details

In [5]:
# For mac machine - we need to get the current machine details for compiling and running the C++ program locally. Program system_info.py is added to current library
from system_info import retrieve_system_info 
system_info=retrieve_system_info()
system_info

{'os': {'system': 'Darwin',
  'arch': 'arm64',
  'release': '25.0.0',
  'version': 'Darwin Kernel Version 25.0.0: Wed Sep 17 21:41:45 PDT 2025; root:xnu-12377.1.9~141/RELEASE_ARM64_T6000',
  'kernel': '25.0.0',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': ''},
 'package_managers': ['xcode-select (CLT)', 'brew', 'port'],
 'cpu': {'brand': 'Apple M1 Pro',
  'cores_logical': 8,
  'cores_physical': 8,
  'simd': []},
 'toolchain': {'compilers': {'gcc': '',
   'g++': 'Apple clang version 17.0.0 (clang-1700.3.19.1)',
   'clang': 'Apple clang version 17.0.0 (clang-1700.3.19.1)',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': 'GNU Make 3.81'},
  'linkers': {'ld_lld': ''}}}

# Create a message for GPT - how to run C++ code on Macbook with configuration listed above

In [6]:
message = f"""
Here is a report of the system information for my computer.
I want to run a C++ compiler to compile a single C++ file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any C++ compiler to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile C++ code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.

System information:
{system_info}
"""

In [7]:
# make a call to open ai with this message
response = openai.chat.completions.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))

You’re already set up. Your system has Apple Clang (C/C++), version 17, via the Xcode Command Line Tools. No installation is needed.

Use these commands in Python:

compile_command = ["clang++", "-std=c++20", "-O3", "-flto=thin", "-mcpu=native", "main.cpp", "-o", "main"]
run_command = ["./main"]

Notes:
- clang++ is the C++ frontend for Apple Clang; g++ on macOS is just a symlink to clang++.
- -O3 and -flto=thin aim for maximum runtime performance; -mcpu=native tunes for your M1 Pro. If you prefer broader compatibility, omit -mcpu=native.

In [8]:
# Copy compile and run command from above
compile_command = ["clang++", "-std=c++20", "-O3", "-flto=thin", "-mcpu=native", "main.cpp", "-o", "main"] 
run_command = ["./main"]

# Main Task - Convert Python program to optimized C++ program

In [9]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

In [10]:
# defining user prompt which contains system info, compile command and python program 
def user_prompt_for(python):
    return f""" Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [11]:
# create message in openai structure 
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [12]:
# write the program to a file 
def write_output(cpp):
    with open("main.cpp", "w", encoding="utf-8") as f:
        f.write(cpp)

In [13]:
# Port method to call api and extract from the response 
def port(client, model, python):
    # set reasoning effort to high 
    reasoning_effort = "high" if 'gpt' in model else None

    # create response from openaai 
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)

    # get the reply 
    reply = response.choices[0].message.content

    # extract program and replace any ```cpp with ``` 
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)

In [14]:
python_program_str="""
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [15]:
# To run python program which is in string format like above 
def run_python(code):
    globals = {"__builtins__": __builtins__}
    exec(code, globals)

In [16]:
run_python(python_program_str)

Result: 3.141592656089
Execution Time: 19.913970 seconds


### send this program to open ai for conversion

In [17]:
port(openai, OPENAI_MODEL, python_program_str)

# Compiling and Executing C++ program

This next cell contains the command to compile a C++ file based on the instructions from GPT.

Again, it's not crucial to do this step if you don't wish to!

OR alternatively: student Sandeep K.G. points out that you can run Python and C++ code online to test it out that way. Thank you Sandeep!  
> Not an exact comparison but you can still get the idea of performance difference.  
> For example here: https://www.programiz.com/cpp-programming/online-compiler/

In [20]:
# Use the commands from GPT 5

def compile_and_run():
    # compile the program 
    subprocess.run(compile_command, check=True, text=True, capture_output=True)

    # run the program 3 times
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)

In [22]:
compile_and_run()

Result: 3.141592656089
Execution Time: 0.395001 seconds

Result: 3.141592656089
Execution Time: 0.388277 seconds

Result: 3.141592656089
Execution Time: 0.391426 seconds

